<a href="https://colab.research.google.com/github/richardachen/Fireproof/blob/main/Wind_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import packages

import ee
from ee import batch
from pandas.tseries.offsets import DateOffset
import pandas as pd

In [2]:
# initialization
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=cTMohA8RB6DnEaCVnXwnwtsb-GX-qfDNRUAYCIGKFns&tc=90hZ87ZXROqrWZAwr5ZFNHbVsWYsMW97H0YxbJkEfWw&cc=rZLaPFNzPX-4afM_PZt4UqBKelY7M4l3GqNVCy0duDI

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgPsAiPY9IgGKaCc3nQ_qZ3YDX6EFbR30y_E9Zsq2hSOb7SL4lywDA

Successfully saved authorization token.


In [10]:
# Woolsey Fire dates
start_date = '2018-11-05'
end_date = '2018-11-24'

# get climate data
gridMET = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")
wind_collection = ee.ImageCollection(gridMET.select('vs')).filterDate(start_date, end_date)

ee.Number({
  "functionInvocationValue": {
    "functionName": "Collection.size",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.map",
                "arguments": {
                  "baseAlgorithm": {
                    "functionDefinitionValue": {
                      "argumentNames": [
                        "_MAPPING_VAR_0_0"
                      ],
                      "body": {
                        "functionInvocationValue": {
                          "functionName": "Image.select",
                          "arguments": {
                            "bandSelectors": {
                              "constantValue": [
                                "vs"
                              ]
                            },
                            "input": {
  

In [4]:
# information to save files later on
folder_name = 'wind_maps'
scale = 250

# Ooo, cool note: "While we need to have an eye on the frequency, 
# GEE takes care of resolution and projection by resampling and 
# reprojecting all data we are going to work with to a common projection"
# https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api

# Woolsey Fire locations
la_lon = -117.9267511
la_lat = 34.2817777
la_poi = ee.Geometry.Point(la_lon, la_lat)
la = la_poi.buffer(1e5) #1e5 meters

In [5]:
# collection = ee.ImageCollection
crs = ee.Image(wind_collection.first()).projection()

def resample(image):
  """
  Resample to new resolution
  """
  image = image.resample('bilinear').reproject(crs=crs, scale=scale)
  return image
if scale != 250:
  wind_collection = wind_collection.map(resample)

In [25]:
n = wind_collection.size().getInfo()
colList = wind_collection.toList(n)

20181107


In [27]:
# export GeoTIFF 
# https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api#save_a_geotiff_file_in_your_google_drive

for i in range(n):
  image = ee.Image(colList.get(i))
  id = image.id().getInfo() or 'image_'+i.toString()
  info_stuff = image.getInfo()
  date = info_stuff['properties']['system:index']

  out = ee.batch.Export.image.toDrive(image=image,
                                      description='wind_speed_in_greater_los_angeles_area',
                                      folder=folder_name,
                                      scale=250,
                                      region=la,
                                      fileNamePrefix='my_export_la_' + date,
                                      crs='EPSG:4326',
                                      fileFormat='GeoTIFF',
                                      maxPixels=1e11)
  batch.Task.start(out)